# setup

connect google drive as sample storage

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

load oridnary libraries

In [1]:
import os

import cv2

load AI libraries

In [2]:
import tensorflow as tf

from tensorflow import keras
from keras import layers
from keras import models
from keras import optimizers

define folders

In [ ]:
base_path = os.path.join('/content/drive/MyDrive/FRAI/')

samples_path = os.path.join(base_path, 'samples')
field_2021_path = os.path.join(samples_path, 'field_2021')
simulation_default_path = os.path.join(samples_path, 'simulation_default')

tmp_path = os.path.join(samples_path, 'tmp')
folds_path = os.path.join(tmp_path, 'folds')
validation_path = os.path.join(tmp_path, 'validation')

#defining training parameters

In [ ]:
amount_of_folds = 10
current_fold = 1

amount_of_samples_from_field_2021 = 0
amount_of_samples_from_simulation_default = 0
#amount_of_samples_from_ = 0

batch_size = 10

# structuring data for training

to be completed

# iterators

training

In [ ]:
def training_generator():
  i = 0
  training_data = numpy.zeros([batch_size, 120, 160])
  training_data_output = numpy.zeros([batch_size, 120, 160])
  while True:
    for i in range(batch_size):
      filename = random.choice(os.listdir(training_data_output_dir))
      img_in = cv2.imread(os.path.join(training_data_dir, filename), cv2.IMREAD_GRAYSCALE)
      # make image b/w
      img_out = cv2.imread(os.path.join(training_data_output_dir, filename), cv2.IMREAD_GRAYSCALE)
      training_data[i] = numpy.asarray(img_in)
      training_data_output[i] = numpy.asarray(img_out)
    training_data = training_data/255.
    training_data_output = training_data_output/255.
    input = training_data.reshape([batch_size, 120, 160, 1])
    label = training_data_output.reshape([batch_size, 120, 160, 1])
    inp = tf.convert_to_tensor(training_data)
    outp = tf.convert_to_tensor(label)
    yield inp, outp

# model

In [ ]:
pool_size = (2, 2)
input_shape = (120, 160, 1)

model = models.Sequential()

# Normalizes incoming inputs. First layer needs the input shape to work
#model.add(layers.normalization.batch_normalization.BatchNormalization(input_shape=input_shape))

# Below layers were re-named for easier reading of model summary; this not necessary
# Conv Layer 1
model.add(layers.Conv2D(8, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv1', input_shape=input_shape))

# Conv Layer 2
model.add(layers.Conv2D(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv2'))

# Pooling 1
model.add(layers.MaxPooling2D(pool_size=pool_size))

# Conv Layer 3
model.add(layers.Conv2D(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv3'))
model.add(layers.Dropout(0.2))

# Conv Layer 4
model.add(layers.Conv2D(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv4'))
model.add(layers.Dropout(0.2))

# Conv Layer 5
model.add(layers.Conv2D(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv5'))
model.add(layers.Dropout(0.2))

# Pooling 2
model.add(layers.MaxPooling2D(pool_size=pool_size))

# Conv Layer 6
model.add(layers.Conv2D(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv6'))
model.add(layers.Dropout(0.2))

# Conv Layer 7
#model.add(layers.Conv2D(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv7'))
#model.add(layers.Dropout(0.2))

# Pooling 3
model.add(layers.MaxPooling2D(pool_size=pool_size))

# Upsample 1
model.add(layers.UpSampling2D(size=pool_size))

# Deconv 1
#model.add(layers.Conv2DTranspose(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv1'))
#model.add(layers.Dropout(0.2))

# Deconv 2
model.add(layers.Conv2DTranspose(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv2'))
model.add(layers.Dropout(0.2))

# Upsample 2
model.add(layers.UpSampling2D(size=pool_size))

# Deconv 3
model.add(layers.Conv2DTranspose(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv3'))
model.add(layers.Dropout(0.2))

# Deconv 4
model.add(layers.Conv2DTranspose(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv4'))
model.add(layers.Dropout(0.2))

# Deconv 5
model.add(layers.Conv2DTranspose(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv5'))
model.add(layers.Dropout(0.2))

# Upsample 3
model.add(layers.UpSampling2D(size=pool_size))

# Deconv 6
model.add(layers.Conv2DTranspose(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv6'))

# Final layer - only including one channel so 1 filter
model.add(layers.Conv2DTranspose(1, (3, 3), padding='valid', strides=(1,1), activation = 'sigmoid', name = 'Final'))

model.summary()

In [ ]:
model.compile(
    loss='mean_squared_error',
    optimizer='Adam',
    metrics=['accuracy']
)

# training & testing

In [ ]:
history = model.fit(
    training_generator(),
    steps_per_epoch=1,
    epochs=500
)

# validation

# export

In [ ]:
model.save('/content/drive/MyDrive/FRAI/model')

# inference